In [75]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn as sk

**This notebook will build a content based recommendation system**

**Importing the datasets**

In [76]:
dfm = pd.read_csv(r"C:\Users\ladap\Downloads\tmdb_5000_movies.csv")
dfc = pd.read_csv(r"C:\Users\ladap\Downloads\tmdb_5000_credits.csv")

In [77]:
#checking the shape of our datasets
print(f'The shape of the movies dataset is {dfm.shape}')
print(f'The shape of the credits dataset is {dfc.shape}')

The shape of the movies dataset is (4803, 20)
The shape of the credits dataset is (4803, 4)


In [78]:
#checking the metadata of the movies dataset
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [79]:
#checking the metadata of the credits dataset
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [80]:
#There are a few null values in both datasets, so we will need to take care of that

In [81]:
#checking the first few rows of our movies dataset
dfm.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [82]:
#checking the first few rows of our credits dataset
dfc.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [83]:
print(dfm.describe())
print(dfc.describe())

             budget             id   popularity       revenue      runtime  \
count  4.803000e+03    4803.000000  4803.000000  4.803000e+03  4801.000000   
mean   2.904504e+07   57165.484281    21.492301  8.226064e+07   106.875859   
std    4.072239e+07   88694.614033    31.816650  1.628571e+08    22.611935   
min    0.000000e+00       5.000000     0.000000  0.000000e+00     0.000000   
25%    7.900000e+05    9014.500000     4.668070  0.000000e+00    94.000000   
50%    1.500000e+07   14629.000000    12.921594  1.917000e+07   103.000000   
75%    4.000000e+07   58610.500000    28.313505  9.291719e+07   118.000000   
max    3.800000e+08  459488.000000   875.581305  2.787965e+09   338.000000   

       vote_average    vote_count  
count   4803.000000   4803.000000  
mean       6.092172    690.217989  
std        1.194612   1234.585891  
min        0.000000      0.000000  
25%        5.600000     54.000000  
50%        6.200000    235.000000  
75%        6.800000    737.000000  
max      

In [84]:
print(dfc.columns)

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')


In [85]:
# 2 features(id=movie_id) are the same in both dataset so we can join them ON this feature
#first, the name of this feature in both datasets must be the same
dfc.rename(columns={"movie_id": "id"}, inplace=True)

In [86]:
dfc.columns

Index(['id', 'title', 'cast', 'crew'], dtype='object')

In [87]:
dfm.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [88]:
#Now we can merge these datasets on this feature
movies = pd.merge(dfm, dfc, on=["id","title"])
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [89]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

**Fixing the null values**

In [90]:
#Filling the null values of 'homepage' with empty strings
#since there is a considerable number of missing values, I will fill them with empty strings
movies['homepage'] = movies['homepage'].fillna('')
movies['tagline'] = movies['tagline'].fillna('')

In [91]:
# 'Overview' has just 3 null values so I will get the values online and insert them

In [92]:
#Getting the info of the 3 overview null values to search the internet for their overview(plot)
null_rows = movies.loc[movies["overview"].isna()]
null_rows

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
2656,15000000,"[{""id"": 18, ""name"": ""Drama""}]",,370980,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",it,Chiamatemi Francesco - Il Papa della gente,NaN,0.738646,"[{""name"": ""Taodue Film"", ""id"": 45724}]",...,0,NaN,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,,Chiamatemi Francesco - Il Papa della gente,7.3,12,"[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4140,2,"[{""id"": 99, ""name"": ""Documentary""}]",,459488,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",en,"To Be Frank, Sinatra at 100",NaN,0.050625,"[{""name"": ""Eyeline Entertainment"", ""id"": 60343}]",...,0,NaN,[],Released,,"To Be Frank, Sinatra at 100",0.0,0,"[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."
4431,913000,"[{""id"": 99, ""name"": ""Documentary""}]",,292539,[],de,Food Chains,NaN,0.795698,[],...,0,83.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,,Food Chains,7.4,8,[],"[{""credit_id"": ""5470c3b1c3a368085e000abd"", ""de..."


In [93]:
#Inserting the missing values of 'Overview'
movies.loc[2656, 'overview'] = 'A biopic of the rise of father Jorge Mario Bergoglio SJ from a teacher in a Jesuit High School in Argentina to archbishop and cardinal of Buenos Aires to Pope of the Roman Catholic Church. The story touches on his relation with his fellow Jesuits in Argentina and Europe, to his relation with laureate writer Jorge Luis Borges, Argentine dictator Jorge Rafael Videla, and archbishops Laghi (nuncio to Argentina) and Quarracino (cardinal of Buenos Aires), up to the moment where he is elected Pope in 2013'
movies.loc[4140, 'overview'] = 'The life of Frank Sinatra, as an actor and singer and the steps along the way that led him to become such an icon.'
movies.loc[4431, 'overview'] = 'To protest their working conditions and poor wages, farmworkers in Immokalee, Florida, start a hunger strike outside the headquarters of Publix supermarkets.'

In [94]:
#Getting the info of the release_date null value to search the internet for it's missing value
null_rows2 = movies.loc[movies["release_date"].isna()]
null_rows2

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
4553,0,[],,380097,[],en,America Is Still the Place,1971 post civil rights San Francisco seemed li...,0.0,[],...,0,0.0,[],Released,,America Is Still the Place,0.0,0,[],[]


In [95]:
#Inserting the missing value of 'release_date'
movies.loc[4553, 'release_date'] ='2013-09-2015'

In [96]:
#Getting the info of the 3 runtime null values to search the internet for their missing values
null_rows3 = movies.loc[movies["runtime"].isna()]
null_rows3

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
2656,15000000,"[{""id"": 18, ""name"": ""Drama""}]",,370980,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",it,Chiamatemi Francesco - Il Papa della gente,A biopic of the rise of father Jorge Mario Ber...,0.738646,"[{""name"": ""Taodue Film"", ""id"": 45724}]",...,0,NaN,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,,Chiamatemi Francesco - Il Papa della gente,7.3,12,"[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4140,2,"[{""id"": 99, ""name"": ""Documentary""}]",,459488,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",en,"To Be Frank, Sinatra at 100","The life of Frank Sinatra, as an actor and sin...",0.050625,"[{""name"": ""Eyeline Entertainment"", ""id"": 60343}]",...,0,NaN,[],Released,,"To Be Frank, Sinatra at 100",0.0,0,"[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."


In [97]:
#Inserting the missing value of 'runtime'
movies.loc[2656, 'runtime'] = 98
movies.loc[4140, 'runtime'] = 81

In [98]:
#checking for any more null values
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              4803 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4803 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4803 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4803 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

**Building the Content based recommendation system**

In [99]:
#Content based recommendation system is based on selecting features of movies or attributes of items to recommend other 
#items that are similar to them. For this dataset we can use the overview, genres, cast, directors, or other characteristics 
#of movies to recommend similar movies to users. The problem is often selecting the right features to use.

**Let's use the overview feature since it gives the plot of the movie. What better feature than the plot?**

In [100]:
movies['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [101]:
#However, the plot is too long so we need to select the most important keywords using TF-IDF

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [103]:
# creating an instance of the TfidfVectorizer that will automatically remove all english stop words such as 'a', 'the'
vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)

In [104]:
# learning the vocabulary of the text data and creating the tf-idf matrix
matrix = vectorizer.fit_transform(movies['overview'])

In [105]:
#checking the shape of the tfidf matrix
matrix.shape

(4803, 20995)

In [106]:
#Since we have the important words from the feature - 'overview', we can compute the similarity between movies, and 
#subsequently find similar movies to recommend to a user.

In [107]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(matrix, matrix)

In [108]:
cosine_similarities

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160548, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160548, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

In [109]:
# I am going to assign indices to the movies to make identification easier
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()
indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [110]:
#Next, I am going to define a function that takes in a movie title it's input and returns a list of 10 most similar movies.
def recommendations(title, cosine_similarities=cosine_similarities):
    # Find the index of the movie that matches the given title
    idx = indices[title]

    # Initialize a list of tuples to store the movie indices and similarity scores
    sim_scores = []

    # Loop over all movies in the dataset
    for i in range(0, len(cosine_similarities[idx])):
        # Get the similarity score for the current movie
        sim = cosine_similarities[idx][i]

        # Add the movie index and similarity score to the list of tuples
        sim_scores.append((i, sim))

    # Sort the movies according to their similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Retrieve the scores of the top 10 movies according to their similarity scores
    sim_scores = sim_scores[1:11]

    # Retrieve the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Provide the top 10 most similar movies
    return movies['title'].iloc[movie_indices]

In [140]:
recommendations('Spider-Man 3')

159                   Spider-Man
30                  Spider-Man 2
1534               Arachnophobia
20        The Amazing Spider-Man
38      The Amazing Spider-Man 2
1318                   The Thing
4664                     Bronson
3610           Not Easily Broken
4456       Raising Victor Vargas
4276                   Def-Con 4
Name: title, dtype: object

In [137]:
recommendations("Harry Potter and the Half-Blood Prince")

114           Harry Potter and the Goblet of Fire
113     Harry Potter and the Order of the Phoenix
191      Harry Potter and the Prisoner of Azkaban
276       Harry Potter and the Chamber of Secrets
501                             The Little Prince
4767                               Run, Hide, Die
197      Harry Potter and the Philosopher's Stone
4133                                        Plush
3015                                   Sinister 2
876                          Domestic Disturbance
Name: title, dtype: object

**This is a great recommendation system, because it uses the plot of a movie to recommend new movies with similar plot. 
#However, this same quality is an issue because it is over-specialized. It recommends only movies with similar plots. 
#What if the user wants movies with different plots as our taste and mood often changes.**

**To solve this problem of overspecificity, I will add new features for consideration by the model. This will make the model learn similarity not just based on plot, but also on other features like cast, director, genre,etc.**

**After going through the data, I will be using the following features for a better recommendation system:
#Top 5 actors, Genre, Keywords, Director, Executive Producer, Producer, Production companies and Vote Average**

In [113]:
a = type(movies['cast'][0])
b = type(movies['crew'][0])
c = type(movies['keywords'][0])
d = type(movies['genres'][0])
e = type(movies['vote_average'][0])
f = type(movies['production_companies'][0])

print(f'Cast is of datatype {a}')
print(f'Crew is of datatype {b}')
print(f'Keywords is of datatype {c}')
print(f'genres is of datatype {d}')
print(f'vote_average is of datatype {e}')
print(f'production_companies is of datatype {f}')

Cast is of datatype <class 'str'>
Crew is of datatype <class 'str'>
Keywords is of datatype <class 'str'>
genres is of datatype <class 'str'>
vote_average is of datatype <class 'numpy.float64'>
production_companies is of datatype <class 'str'>


In [114]:
#because the datatype of some of the features we are interested in are strings in a list, I would have to convert them to 
#lists, so they can be used. 
# importing the `json` module
import json

# creating a list of the feature names that need conversion
features = ['cast', 'crew', 'keywords', 'genres', 'production_companies']

# iterate over the list of feature names
for feature in features:
    # apply the `json.loads()` function to each feature column in the DataFrame
    movies[feature] = movies[feature].apply(json.loads)

In [115]:
#Checking the type of the features we are interested in to confirm the conversion
a = type(movies['cast'][0])
b = type(movies['crew'][0])
c = type(movies['keywords'][0])
d = type(movies['genres'][0])
e = type(movies['vote_average'][0])
f = type(movies['production_companies'][0])

print(f'Cast is of datatype {a}')
print(f'Crew is of datatype {b}')
print(f'Keywords is of datatype {c}')
print(f'genres is of datatype {d}')
print(f'vote_average is of datatype {e}')
print(f'production_companies is of datatype {f}')

Cast is of datatype <class 'list'>
Crew is of datatype <class 'list'>
Keywords is of datatype <class 'list'>
genres is of datatype <class 'list'>
vote_average is of datatype <class 'numpy.float64'>
production_companies is of datatype <class 'list'>


In [116]:
#Now, I will extract the Director's name, and add it to the dataset
movies['director'] = movies['crew'].apply(lambda x: next((i["name"] for i in x if i["job"] == "Director"), np.nan))

In [117]:
#Previewing the feature(director)
movies['director']

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4          Andrew Stanton
              ...        
4798     Robert Rodriguez
4799         Edward Burns
4800          Scott Smith
4801          Daniel Hsia
4802     Brian Herzlinger
Name: director, Length: 4803, dtype: object

In [118]:
#Now, I will extract the Director's name, and add it to the dataset
movies['director'] = movies['crew'].apply(lambda x: next((i["name"] for i in x if i["job"] == "Director"), np.nan))

In [119]:
#Now, I will extract the Writer's name, and add it to the dataset
movies['writer'] = movies['crew'].apply(lambda x: next((i["name"] for i in x if i["job"] == "Writer"), np.nan))

In [120]:
#Now, I will extract the Producers' name, and add it to the dataset
def get_producers(data, return_all):
    # Get a list of all producers
    producers = [i["name"] for i in data if i["job"] == "Producer"]
    
    if len(producers) == 0:
        # If there are no producers, return NaN
        return np.nan

    if len(producers) > 1 or return_all:
        # If there are multiple producers or the `return_all` flag is set, return all producers
        return producers
    else:
        # Otherwise, return the first producer
        return next((i["name"] for i in data if i["job"] == "Producer"), np.nan)

# Since I want it to return all producers, I will set the return all as True)
movies['producers'] = movies['crew'].apply(lambda x: get_producers(x, return_all=True))

In [121]:
#Now, I will extract the Executive Producer names, and add it to the dataset
def get_executive_producers(data, return_all):
    # Get a list of all producers
    executive_producers = [i["name"] for i in data if i["job"] == "Executive Producer"]
    
    if len(executive_producers) == 0:
        # If there are no producers, return NaN
        return np.nan

    if len(executive_producers) > 1 or return_all:
        # If there are multiple producers or the `return_all` flag is set, return all producers
        return executive_producers
    else:
        # Otherwise, return the first producer
        return next((i["name"] for i in data if i["job"] == "Executive Producer"), np.nan)

# Since I want it to return all producers, I will set the return all as True)
movies['executive_producers'] = movies['crew'].apply(lambda x: get_executive_producers(x, return_all=True))

In [122]:
#Now, I will extract the Production company names, and add it to the dataset
movies['production_companies_names'] = movies['production_companies'].apply(lambda x: [production_companies['name'] for production_companies in x])

In [123]:
#In the 'cast','keywords','genres','crew',production companies features, there are multiple names. So, I am going to select the top 5 in each 
#category by using a function
def top_5(x):
    names = []
    if isinstance(x, list):
        for i in x:
            if 'name' in i:
                names.append(i['name'])
        #Return the first three elements of the list. If less than or equal to three, return all elements
        if len(names) > 5:
            names = names[:5]
        return names

    #In the case where no valid names are found in the input list, or the input is not a list at all,
    # return an empty list
    return names
features = ['cast', 'keywords', 'genres']
for feature in features:
    movies[feature] = movies[feature].apply(top_5)

In [124]:
#printing the new dataframe that I will be feeding the model
movies[['title', 'cast', 'director', 'keywords', 'genres','writer','producers', 'executive_producers','production_companies_names','vote_average']].head()

,title,cast,director,keywords,genres,writer,producers,executive_producers,production_companies_names,vote_average
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]",James Cameron,"[James Cameron, Jon Landau]",[Laeta Kalogridis],"[Ingenious Film Partners, Twentieth Century Fo...",7.2
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]",NaN,"[Jerry Bruckheimer, Eric McLeod, Chad Oman, Pe...",[Mike Stenson],"[Walt Disney Pictures, Jerry Bruckheimer Films...",6.9
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes,"[spy, based on novel, secret agent, sequel, mi6]","[Action, Adventure, Crime]",NaN,"[Barbara Broccoli, Michael G. Wilson]",[Callum McDougall],"[Columbia Pictures, Danjaq, B24]",6.3
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",NaN,"[Charles Roven, Christopher Nolan, Emma Thomas]","[Michael Uslan, Benjamin Melniker, Thomas Tull...","[Legendary Pictures, Warner Bros., DC Entertai...",7.6
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]",NaN,"[Colin Wilson, Jim Morris, Lindsey Collins]",NaN,[Walt Disney Pictures],6.1


In [125]:
#Some features might have similar or repeating words such as genres, first name, last name, etc. so I am going to strip all 
#spaces and convert all strings to lower case
def format_data(x):
    #checking if the input x is a list.If yes, then carry out the below code
    if isinstance(x, list):
        return [i.strip().lower() for i in x]
    else:
        #Checking if the input x is a string. If yes, then carry out the below code
        if isinstance(x, str):
            return x.strip().lower()
        else:
            return ''

features = ['cast', 'director', 'keywords', 'genres','writer','producers', 'executive_producers','production_companies_names']

for feature in features:
    movies[feature] = movies[feature].apply(format_data)

In [126]:
#Now, I will join all the features for each record together to create a string that will be fed into a vectorizer
def concat(x):
    return ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['keywords']) + ' ' + ' '.join(x['genres'])\
     + ' ' + x['writer'] + ' ' + ' '.join(x['producers']) + ' ' + ' '.join(x['executive_producers'])\
    + ' ' + ' '.join(x['production_companies_names']) + ' ' + str(x['vote_average'])
movies['concat_string'] = movies.apply(concat, axis=1) 

In [127]:
#previewing the movies dataset with the extracted features
movies[[ 'cast', 'director', 'keywords', 'genres','writer','producers', 'executive_producers','production_companies_names','vote_average']].head()

,cast,director,keywords,genres,writer,producers,executive_producers,production_companies_names,vote_average
0,"[sam worthington, zoe saldana, sigourney weave...",james cameron,"[culture clash, future, space war, space colon...","[action, adventure, fantasy, science fiction]",james cameron,"[james cameron, jon landau]",[laeta kalogridis],"[ingenious film partners, twentieth century fo...",7.2
1,"[johnny depp, orlando bloom, keira knightley, ...",gore verbinski,"[ocean, drug abuse, exotic island, east india ...","[adventure, fantasy, action]",,"[jerry bruckheimer, eric mcleod, chad oman, pe...",[mike stenson],"[walt disney pictures, jerry bruckheimer films...",6.9
2,"[daniel craig, christoph waltz, léa seydoux, r...",sam mendes,"[spy, based on novel, secret agent, sequel, mi6]","[action, adventure, crime]",,"[barbara broccoli, michael g. wilson]",[callum mcdougall],"[columbia pictures, danjaq, b24]",6.3
3,"[christian bale, michael caine, gary oldman, a...",christopher nolan,"[dc comics, crime fighter, terrorist, secret i...","[action, crime, drama, thriller]",,"[charles roven, christopher nolan, emma thomas]","[michael uslan, benjamin melniker, thomas tull...","[legendary pictures, warner bros., dc entertai...",7.6
4,"[taylor kitsch, lynn collins, samantha morton,...",andrew stanton,"[based on novel, mars, medallion, space travel...","[action, adventure, science fiction]",,"[colin wilson, jim morris, lindsey collins]",,[walt disney pictures],6.1


In [128]:
#previewing a record of the concatenated string
movies['concat_string'][0]

'sam worthington zoe saldana sigourney weaver stephen lang michelle rodriguez james cameron culture clash future space war space colony society action adventure fantasy science fiction james cameron james cameron jon landau laeta kalogridis ingenious film partners twentieth century fox film corporation dune entertainment lightstorm entertainment 7.2'

In [129]:
#Now, I will convert the concatenated strings into a numerical representation using CountVectorizer. Note that I am using
#CountVectorizer in this situation, not TF-IDF because I want it to treat every string equally important since every string
#here is equally important. 
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer(stop_words='english')
count_matrix = count_vector.fit_transform(movies['concat_string'])

In [130]:
#computing the cosine similarity matrix for the count_matrix
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities_2 = cosine_similarity(count_matrix, count_matrix)

In [131]:
#Now, I am going to reset the index, so I can create a new map to find movie titles and vice versa
movies = movies.reset_index()
indices = pd.Series(movies.index, index=movies['title'])
indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [141]:
#Now, I can use the defined function- recommendation with the new cosine similarity to generate better recommendations
recommendations("Spider-Man 3", cosine_similarities_2)

30                  Spider-Man 2
159                   Spider-Man
20        The Amazing Spider-Man
165                         Hulk
162                      Stealth
1547                  The Butler
38      The Amazing Spider-Man 2
68                      Iron Man
668                      Elektra
232                The Wolverine
Name: title, dtype: object

In [133]:
recommendations("Harry Potter and the Half-Blood Prince")


114           Harry Potter and the Goblet of Fire
113     Harry Potter and the Order of the Phoenix
191      Harry Potter and the Prisoner of Azkaban
276       Harry Potter and the Chamber of Secrets
501                             The Little Prince
4767                               Run, Hide, Die
197      Harry Potter and the Philosopher's Stone
4133                                        Plush
3015                                   Sinister 2
876                          Domestic Disturbance
Name: title, dtype: object

**Looking at the new result, we have better recommendations that takes into account keywords, genres,cast, director, writer,
producers, executive producers, production companies and even average vote.This gives a much more accurate prediction than 
just using the movie's synopsis**

**Evaluation of the two methods**

In [162]:
#I am going to write a function to output the similarity scores of the movies recommended given the titles
def cos_sim(title1, titles2, cosine_similarities=cosine_similarities):
    # Find the index of the movie that matches the given title
    idx1 = indices[title1]

    # Initialize a list of tuples to store the movie indices and similarity scores
    sim_scores = []

    # Loop over all movies in the list of titles
    for title2 in titles2:
        # Find the index of the current movie
        idx2 = indices[title2]

        # Compute the cosine similarity value between the two movies
        sim = cosine_similarities[idx1][idx2]

        # Add the movie index and similarity score to the list of tuples
        sim_scores.append((idx2, sim))

    # Sort the movies according to their similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Retrieve the scores of the top 10 movies according to their similarity scores
    sim_scores = sim_scores[1:11]

    # Retrieve the movie indices and similarity scores
    movie_indices = [i[0] for i in sim_scores]
    sim_scores = [i[1] for i in sim_scores]

    # Provide the top 10 most similar movies along with their similarity scores
    return list(zip(movies['title'].iloc[movie_indices], sim_scores))

In [155]:
#Movies recommended to be similar to Spider-man 3 by the model built on only 'overview'
result_1 = recommendations("Spider-Man 3", cosine_similarities)
result_1

159                   Spider-Man
30                  Spider-Man 2
1534               Arachnophobia
20        The Amazing Spider-Man
38      The Amazing Spider-Man 2
1318                   The Thing
4664                     Bronson
3610           Not Easily Broken
4456       Raising Victor Vargas
4276                   Def-Con 4
Name: title, dtype: object

In [173]:
result_1_list = result_1.to_list()
result_1_list

['Spider-Man',
 'Spider-Man 2',
 'Arachnophobia',
 'The Amazing Spider-Man',
 'The Amazing Spider-Man 2',
 'The Thing',
 'Bronson',
 'Not Easily Broken',
 'Raising Victor Vargas',
 'Def-Con 4']

In [172]:
#Movies recommended to be similar to Spider-man 3 by the model built on 8 features
result_2= recommendations("Spider-Man 3", cosine_similarities_2)
result_2

30                  Spider-Man 2
159                   Spider-Man
20        The Amazing Spider-Man
165                         Hulk
162                      Stealth
1547                  The Butler
38      The Amazing Spider-Man 2
68                      Iron Man
668                      Elektra
232                The Wolverine
Name: title, dtype: object

In [174]:
result_2_list = result_2.to_list()
result_2_list

['Spider-Man 2',
 'Spider-Man',
 'The Amazing Spider-Man',
 'Hulk',
 'Stealth',
 'The Butler',
 'The Amazing Spider-Man 2',
 'Iron Man',
 'Elektra',
 'The Wolverine']

In [167]:
scores_1 = cos_sim("Spider-Man 3",result_1_list, cosine_similarities)
scores_1

[('Spider-Man 2', 0.24647209184826271),
 ('Arachnophobia', 0.2367104604521435),
 ('The Amazing Spider-Man', 0.22285177568876832),
 ('The Amazing Spider-Man 2', 0.21140076729082483),
 ('The Thing', 0.17166864148468536),
 ('Bronson', 0.10119023772910767),
 ('Not Easily Broken', 0.09698331013627295),
 ('Raising Victor Vargas', 0.09441340533091117),
 ('Def-Con 4', 0.09420458163538706)]

In [168]:
scores_2 = cos_sim("Spider-Man 3",result_2_list, cosine_similarities_2)
scores_2

[('Spider-Man', 0.5586203877309283),
 ('The Amazing Spider-Man', 0.46318355658658544),
 ('Hulk', 0.31580697039994465),
 ('Stealth', 0.30942637387763805),
 ('The Butler', 0.29638141859562284),
 ('The Amazing Spider-Man 2', 0.2859525795719834),
 ('Iron Man', 0.280285101520231),
 ('Elektra', 0.25434691255526276),
 ('The Wolverine', 0.25136634999228946)]

In [170]:
sim_scores_1 = pd.DataFrame(scores_1)
sim_scores_1

,0,1
0,Spider-Man 2,0.246472
1,Arachnophobia,0.236710
2,The Amazing Spider-Man,0.222852
3,The Amazing Spider-Man 2,0.211401
4,The Thing,0.171669
5,Bronson,0.101190
6,Not Easily Broken,0.096983
7,Raising Victor Vargas,0.094413
8,Def-Con 4,0.094205


In [171]:
sim_scores_2 = pd.DataFrame(scores_2)
sim_scores_2

,0,1
0,Spider-Man,0.558620
1,The Amazing Spider-Man,0.463184
2,Hulk,0.315807
3,Stealth,0.309426
4,The Butler,0.296381
5,The Amazing Spider-Man 2,0.285953
6,Iron Man,0.280285
7,Elektra,0.254347
8,The Wolverine,0.251366
